In [19]:
import numpy as np

In [20]:
BATCH_SIZE = 50000
EPOCHS = 500

In [21]:
# Load train, validation, and testing data from saved numpy arrays

x_train = np.load('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/x_train.npy')
y_train = np.load('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/y_train.npy')
x_val = np.load('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/x_val.npy')
y_val = np.load('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/y_val.npy')
x_test = np.load('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/x_test.npy')
y_test = np.load('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/y_test.npy')

print("X train shape: {}\nY_train shape: {}\nX_val shape: {}\nY_val shape: {}\
      \nX test shape: {}\nY test shape: {}".format(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape))

X train shape: (1482192, 8, 1)
Y_train shape: (1482192, 2)
X_val shape: (494064, 8, 1)
Y_val shape: (494064, 2)      
X test shape: (494064, 8, 1)
Y test shape: (494064, 2)


In [22]:
from keras.layers import Input
from keras.models import Model
from keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.optimizers import Adam

def cnn_model():
    inp = Input(shape=x_train.shape[1:])
    conv1 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(inp)
    conv1 = BatchNormalization(axis=-1, center=True, scale=True)(conv1)
    conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(conv1)
    conv2 = BatchNormalization(axis=-1, center=True, scale=True)(conv2)
    conv3 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(conv2)
    conv3 = BatchNormalization(axis=-1, center=True, scale=True)(conv3)
    pool1 = MaxPooling1D(pool_size=2)(conv3)
    flattern = Flatten()(pool1)
    dense1 = Dense(500, activation='relu')(flattern)
    classifier = Dense(2, activation='sigmoid')(dense1)
    
    model = Model(inputs=inp, outputs=classifier)
    
    opt = Adam(lr=1e-6)
    model.compile(loss='mse', optimizer=opt, metrics=['mean_squared_error'])
    
    return model

In [23]:
callbacks = [ModelCheckpoint(
        filepath='/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Checkpoints/checkpoint.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True), EarlyStopping(monitor='val_mean_squared_error', patience=50)]

In [27]:
model = cnn_model()
print(model.summary())

# Train the regressor
# model.fit(x_train, y_train, validation_data=(x_val, y_val), verbose=1, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 8, 1)              0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 8, 32)             128       
_________________________________________________________________
batch_normalization_4 (Batch (None, 8, 32)             128       
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 8, 64)             6208      
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 64)             256       
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 8, 128)            24704     
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 128)            512       
__________

In [26]:
from keras.models import load_model
load_model('Checkpoints/checkpoint.50-0.01.h5')
# Evaluate model with test data
model.evaluate(x_test, y_test)

494064/494064 [==============================] - 27s 55us/step


[0.004024454036856574, 0.004024454036856574]